This notebook offers tricks for diagnosing problems in the optimization and debugging them

## Finding infeasible pairs

What do you do when the model solver returns with "Infeasible model"?

Consider using the `findinfeasiblepair(house, solver)` tool.  It will give you a range within the matrix for which the results become minimally infeasible.  In other words, suppose that the full linear programming matrix is $A$.  It returns $i$, $j$, such that $A[1:i, :]$ is infeasible, but $A[1:i-1, :]$ is not, and $A[j:end, :]$ is infeasible but $A[j+1:end, :]$ is not.

You can then investigate individual rows by calling `house.A[row,:]`.

Note that the linear programming problem is always solved as follows:
$$
\max_x f' x
$$
s.t. $A x < b$ and $x > 0$.

# Breaking apart the matrix

The entries in the $f$ vector correspond to a sequences of parameter values, those of the $b$ vector correspond to variable values, and the rows and columns of $A$ correspond to variables and parameters respectively.

To find out which parameter a particular column of $A$ or value in $f$ corresponds to, use
```
cumsum(varlengths(house.model, house.paramcomps, house.parameters))
```
which gives the **last index** of each paramter.

To find out which variable a particular row of $A$ or value in $b$ corresponds to, use
```
cumsum(varlengths(house.model, house.constcomps, house.constraints, house.constdictionary))
```
which gives the **last index** of each variable.